# to do : detect subtitle

In [ ]:
import sys, os, git, rstparse
import numpy as np

In [2]:
from utilities import detect_block, block_to_sentence, fix_link, fix_caption, \
                      fix_italic, replace_special_character, fix_math

In [3]:
current_path = os.getcwd()
git_repo = git.Repo(current_path, search_parent_directories=True)
git_path = git_repo.git.rev_parse("--show-toplevel")
sys.path.append(git_path+"/docs/inputs/shared-pyplot-files/")

In [4]:
tutorials = {"level0": ["lennard-jones-fluid"], }
             #"level1": ["breaking-a-carbon-nanotube"],
             #"level2": ["polymer-in-water", "nanosheared-electrolyte"],
             #"level3": ["water-adsorption-in-silica", "free-energy-calculation", "reactive-silicon-dioxide"]}

In [5]:
class RSTtoTEX:
    """Read RST file and convert it to TEX file."""
    def __init__(self, file_name, *args, **kwargs,):
        """Initialize"""
        super().__init__(*args, **kwargs)
        self.file_name = file_name

    def convert_file(self):
        """Main convert function."""
        self.read_rst()
        self.detect_blocks()
        self.detect_title()
        self.detect_label()
        self.detect_hatnote()

    def read_rst(self):
        """Convert the rst file into a list of strings"""
        rst = rstparse.Parser()
        with open(self.file_name) as f:
            rst.read(f)
        rst.parse()
        file_content = []
        for line in rst.lines:
            file_content.append(line)
        self.file_content = file_content

    def detect_blocks(self):
        """Detect the block structure of the file"""
        jumps = []
        block_position = []
        block_type = []
        for n, line in enumerate(self.file_content):
            jump = 0
            for letter in line:
                if letter == ' ':
                    jump += 1
                else:
                    break
            jumps.append(jump)
            if ('..' in line) & ('...' not in line):
                part = line.split('..')
                #if 'raw:: html' not in part[1]: # exclude link
                    # block type
                if 'figure' in part[1]:
                    block_type.append('figure')
                elif 'raw:: html' in part[1]:
                    block_type.append('link')
                elif 'label' in part[1]:
                    block_type.append('label')
                elif 'justify' in part[1]:
                    block_type.append('justify')
                elif 'lammps' in part[1]:
                    block_type.append('lammps')
                elif 'bw' in part[1]:
                    block_type.append('bw')
                elif 'hatnote' in part[1]:
                    block_type.append('hatnote')
                elif 'admonition' in part[1]:
                    block_type.append('admonition')
                else:
                    block_type.append('None')
                # block position (indentation level)
                if part[0] == '':
                    block_position.append(1)
                elif part[0] == '    ':
                    block_position.append(2)
                else:
                    print('WARNING: UNKNOWN BLOCK TYPE')
                    block_position.append(0)
                #else:
                #    block_position.append(0)
                #    block_type.append('None')
            else:
                block_position.append(0)
                block_type.append('None')
        self.jumps = jumps
        self.block_position = block_position
        self.block_type = block_type

    def detect_title(self):
        self.detect_title_position()
        if len(self.position_title) > 1:
            print("More than one title was found in", self.title)
        self.title = self.file_content[self.position_title[0]]

    def detect_title_position(self):
        self.position_title = []
        for n, line in enumerate(self.file_content):
            if line[:3] == "***":
                self.position_title.append(n-1)

    def detect_label(self):
        assert len(np.where(np.array(self.block_type) == 'label')[0]) == 1, """Several labels detected"""
        line = self.file_content[np.where(np.array(self.block_type) == 'label')[0][0]]
        self.label = line.split()[1][1:-1]

    def detect_hatnote_position(self):
        self.position_hatnote = []
        for n, line in enumerate(self.file_content):
            if len(line.split()) == 3:
                if (line.split()[0] == '..') & (line.split()[1][:-2] == "container") & (line.split()[2] == "hatnote"):
                    self.position_hatnote.append(n)

    def detect_hatnote(self):
        self.detect_hatnote_position()
        if len(self.position_hatnote) > 1:
            print("More than one hatnote was found in", self.title)
        n = self.position_hatnote[0]
        line = detect_block(n+1, self.file_content)
        self.hatnote = block_to_sentence(line)[0]

In [6]:
for level in tutorials.keys():
    for tutorial in tutorials[level]:
        rst_file_name = git_path+'/docs/sphinx/source/tutorials/'+level+'/'+tutorial+'.rst'
        tex_file_name = git_path+'/ebook/tutorials/'+level+'/'+tutorial+'.tex'
        if os.path.exists(git_path+'/ebook/tutorials') is False:
            os.mkdir(git_path+'/ebook/tutorials')
        if os.path.exists(git_path+'/ebook/tutorials/'+level) is False:
            os.mkdir(git_path+'/ebook/tutorials/'+level)
        RST = RSTtoTEX(file_name = rst_file_name)
        RST.convert_file()
        assert len(RST.block_position) == len(RST.file_content)
        assert len(RST.block_type) == len(RST.file_content)
        assert len(RST.jumps) == len(RST.file_content)


[0, 3, 0, 0]



In [13]:
f = open(tex_file_name, "w") 
f.write('\chapter{'+RST.title+'}')
f.write('\n') 
f.write('\label{'+RST.label+'}')
f.write('\n\n')
f.write(r'\noindent \textit{'+RST.hatnote+'}')
f.write('\n\n')
n = 0
for block_position, block_type in zip(RST.block_position, RST.block_type):
    if block_position > 0:
        block = []
        end_of_block = False
        for m in np.arange(n+1, len(RST.file_content)):
            if ((RST.block_position[m] > block_position) | (RST.block_position[m] == 0)) & (end_of_block is False):
                block.append(m)
            else:
                end_of_block = True

        if block_type == 'justify':
            cpt = 0
            for m in block:
                line = RST.file_content[m]
                try:
                    while line[0] == ' ':
                        line = line[1:]
                except:
                    pass
                line = fix_link(RST, line)
                line = replace_special_character(line, '#', r'$\#$')
                line = fix_italic(line, replace_underscore=True)
                line = fix_math(line)
                if len(line)>0:
                    if cpt == 0:
                        f.write('\n')
                        f.write(r'\vspace{0.5cm} \noindent ' + line)
                    else:
                        f.write(line)
                    cpt += 1
                    f.write('\n')
        else:
            pass
    n += 1
f.close()

In [8]:
stop stop stop

f = open(tex_file_name, "w")
for n in range(len(RST.file_content)):
    if n in RST.position_title:
        line = RST.title
        f.write('\chapter{'+line+'}')
        f.write('\n') 
        line = RST.label
        f.write('\label{'+line+'}')
        f.write('\n\n')
        line = RST.hatnote
        f.write(r'\noindent \textit{'+line+'}')
        f.write('\n\n')
    elif n in RST.position_paragraph:
        lines = RST.paragraphs[np.where(np.array(RST.position_paragraph) == n)[0][0]]
        for line in lines:
            line = fix_link(RST, line)
            line = replace_special_character(line, '#', r'$\#$')
            line = fix_italic(line, replace_underscore=True)
            line = fix_math(line)
            f.write(line)
            f.write('\n\n')
    elif n in RST.position_code:
        lines = RST.codes[np.where(np.array(RST.position_code) == n)[0][0]]
        caption, lines = fix_caption(lines)
        if caption is not None:
            f.write(r'\textit{'+caption+'}'+'\n')
        f.write(r'\begin{verbatim}'+'\n')
        for line in lines:
            f.write(line)
            f.write('\n')
        f.write(r'\end{verbatim}'+'\n\n')
f.close()

SyntaxError: invalid syntax (362998474.py, line 1)

In [ ]:
stop stop stop


def loop_through_file(self):
    """Loop over the rst file and extract the positions of items"""
    self.position_title = []
    self.position_labels = []
    self.position_hatnote = []
    self.position_paragraph = []
    self.position_code = []
    for n, line in enumerate(self.file_content):
        self.detect_title_position(n, line)
        self.detect_label_position(n, line)
        self.detect_hatnote_position(n, line)
        self.detect_paragraph_position(n, line)
        self.detect_code_position(n, line)
    self.detect_title()
    self.detect_label()
    self.detect_hatnote()
    self.detect_paragraph()
    self.detect_code()

def detect_label_position(self, n, line):
    if len(line.split()) == 2:
        if (line.split()[0] == '..') & (line.split()[1][0] == "_"):
            self.position_labels.append(n)

def detect_label(self):
    if len(self.position_labels) > 1:
        print("More than one label was found in", self.title)
    line = self.file_content[self.position_labels[0]]
    self.label = line.split()[1][1:-1]

def detect_hatnote_position(self, n, line):
    if len(line.split()) == 3:
        if (line.split()[0] == '..') & (line.split()[1][:-2] == "container") & (line.split()[2] == "hatnote"):
            self.position_hatnote.append(n)

def detect_hatnote(self):
    if len(self.position_hatnote) > 1:
        print("More than one hatnote was found in", self.title)
    n = self.position_hatnote[0]
    line = detect_block(n+1, self.file_content)
    self.hatnote = block_to_sentence(line)[0]

def detect_paragraph_position(self, n, line):
    if len(line.split()) == 3:
        if (line.split()[0] == '..') & (line.split()[1][:-2] == "container") & (line.split()[2] == "justify"):
            self.position_paragraph.append(n)

def detect_paragraph(self):
    paragraphs = []
    for n in self.position_paragraph:
        paragraph = detect_block(n+1, self.file_content)
        paragraphs.append(block_to_sentence(paragraph))
    self.paragraphs = paragraphs

def detect_code_position(self, n, line):
    if len(line.split()) == 3:
        if (line.split()[0] == '..') & (line.split()[1][:-2] == "code-block"):
            self.position_code.append(n)

def detect_code(self):
    codes = []
    for n in self.position_code:
        code = detect_block(n+1, self.file_content, keep_line_break = True)        
        codes.append(block_to_sentence(code))
    self.codes = codes

stop stop stop

SyntaxError: invalid syntax (1715528835.py, line 1)